In [4]:
import pyscreenshot as ImageGrab
import keras

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is enabled with initial size: 65.0% of memory, cuDNN not available)


### model

In [ ]:
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from itertools import chain
from keras import backend as K
from keras.layers import Dense, Activation, Convolution2D, Convolution3D, MaxPooling2D, MaxPooling3D, BatchNormalization, Flatten, ZeroPadding2D, Dropout
from keras.models import load_model, Sequential
from keras.optimizers import Adam

INPUT_SHAPE = (3, 455, 590)

def center_normalize(x):
    """Custom activation for online sample-wise center and std. normalization."""
    return (x - K.mean(x)) / K.std(x)

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(Activation(activation=center_normalize, input_shape=INPUT_SHAPE))
    model.add(ZeroPadding2D((1,1), input_shape=INPUT_SHAPE))
    # model.add(ZeroPadding2D((1,1),input_shape=(2, INPUT_ROW, INPUT_COL)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-5), loss=OBJECTIVE_FUNCTION, metrics=METRICS)

    return model

model = VGG_16()
# model.summary()

### get screenshot

In [4]:
import numpy as np
import pyscreenshot as ImageGrab
im=ImageGrab.grab(bbox=(120,75,710,530)) # X1,Y1,X2,Y2
# im.show()
im.save("vaca.png")
pix = np.array(im)
print(type(pix))
print(pix.shape)

<class 'numpy.ndarray'>
(455, 590, 3)


### joystick

In [13]:
with open('capture.out', 'rb') as f:
    my_list = pickle.load(f)
    print(my_list)

FileNotFoundError: [Errno 2] No such file or directory: 'capture.out'

## this is q-learning or a trial

In [1]:
from qlearning4k.games.game import Game
from pymouse import PyMouse
from pykeyboard import PyKeyboard

class Mario(Game):
    def __init__(self):
        self.reset()
        self.k = PyKeyboard()

    @property
    def name(self):
        return "Game"

    @property
    def nb_actions(self):
        return 7

    def reset(self):
        pass

    def play(self, action):
        print('this is action: ' + action)
        self.k.tap_key(chr(action))
        
        im = ImageGrab.grab(bbox=(120, 75, 710, 530))
        self.state = im

    def get_state(self):
        self.state

    def get_score(self):
        return 0

    def is_over(self):
        return False

    def is_won(self):
        return False

    def get_frame(self):
        return self.get_state()

    def draw(self):
        return self.get_state()

    def get_possible_actions(self):
        return range(self.nb_actions)

In [3]:
from pymouse import PyMouse
from pykeyboard import PyKeyboard

m = PyMouse()
k = PyKeyboard()

# pressing a key
k.release_key('v')